In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import shapiro, kstest, probplot
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

ModuleNotFoundError: No module named 'sklearn'

## Carga dos dados

In [ ]:
df_pontuacao = pd.read_csv('./datasets/pontuacao_teste.csv')

: 

In [ ]:
df_pontuacao.info()

: 

In [ ]:
# ver Dataframe
df_pontuacao

: 

## EDA

In [ ]:
# Medidas estatísticas das variáveis
df_pontuacao.describe()

: 

In [ ]:
# plot de dispersão
# X = horas_estudo
# y = pontuacao_teste
sns.scatterplot(data=df_pontuacao, x='horas_estudo', y='pontuacao_teste')

: 

In [ ]:
# verificar outliers
# box plot
sns.boxplot(df_pontuacao, y='horas_estudo')

: 

In [ ]:
# verificar outliers
# box plot
sns.boxplot(df_pontuacao, y='pontuacao_teste')

: 

CONCLUSÃO: sem outliers

In [ ]:
# verificar correlação entre variaveis - Pearson
sns.heatmap(df_pontuacao.corr('pearson'), annot=True)

: 

In [ ]:
sns.heatmap(df_pontuacao.corr('spearman'), annot=True)

: 

- não temos outliers
- a dispersão é linear, até certo ponto de 600 de pontuação
- correlação forte

In [ ]:
# histograma
sns.displot(df_pontuacao, x='horas_estudo')

: 

In [ ]:
# histograma
sns.displot(df_pontuacao, x='pontuacao_teste')

: 

## Treinar modelo

In [ ]:
# divisão do dataset entre treino e teste, com ajuste de "shape", pois temos apenas uma feature (coluna) sendo atribuída às variáveis 
X = df_pontuacao.horas_estudo.values.reshape(-1, 1)
y = df_pontuacao.pontuacao_teste.values.reshape(-1, 1)
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

: 

In [ ]:
# instanciar o modelo a ser treinado
reg_model = LinearRegression()

: 

In [ ]:
# treinar o modelo
reg_model.fit(X_train, y_train)

: 

In [ ]:
reg_model.coef_

: 

In [ ]:
reg_model.intercept_

: 

In [ ]:
# imprimir equação da reta
# y = aX + b
print("A equação da reta é y = {:4f}x + {:4f}".format(reg_model.coef_[0][0], reg_model.intercept_[0]))

: 

Ou seja, na reta, onde x é 0, o y é 8.068

## Validação de modelo - métricas

In [ ]:
# predição dos valores com base no conjunto de testes
y_pred = reg_model.predict(x_test)

: 

In [ ]:
# calcular métrica R-squared ou coeficiente de determinação (vai de 0 a 1)
# r^2 representa a proporção da variação na variável dependente, que é explicada pela variável independente
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
r2_score(y_test, y_pred)

: 

In [ ]:
# calcular métrica MAE
# média da diferença entre o valor predito e o valor de teste
# MAE é menos sensível a outliers, por não elevar ao quadrado
mean_absolute_error(y_test, y_pred)

: 

considerando que o teste vá de 0 a 800, 19 pontos de erro na predição talvez seja aceitável

In [ ]:
# calcular métrica MSE
# MSE = média (y_test - y_pred)2
# é mais sensível a outliers e penaliza grandes erros
# é boa para comparação de modelos
mean_squared_error(y_test, y_pred)

: 

In [ ]:
# calcular métrica RMSE
# MSE = Raiz (média (y_test - y_pred)2)
# é mais sensível a outliers e penaliza grandes erros
# é boa para comparação de modelos
mean_squared_error(y_test, y_pred, squared=False)

: 

In [ ]:
# análise gráfica
x_axis = range(len(y_test))
plt.figure(figsize=(10,6))

sns.scatterplot(x=x_axis, y=y_test.reshape(-1), color='blue', label='valores reais')
sns.scatterplot(x=x_axis, y=y_pred.reshape(-1), color='red', label='valores preditos')
plt.legend()
plt.show()

: 

## Análise de resíduos

In [ ]:
# calcular os resíduos
residuos = y_test - y_pred

: 

In [ ]:
# calcular os resíduos de forma padronizada (standardization)
# para cada elemento de um conjunto (X - media) / desvio_padrao
from scipy.stats import zscore
residuos_std = zscore(residuos)

: 

In [ ]:
# verificar linearidade do modelo:
# se os resíduos estiverem entre -2 e +2 (na escala padrão) -> indica linearidade

# verificar homogeneidade das variâncias (homocedasticidade)
# se os valores estiverem em torno da reta, temos homocedasticidade, caso contrário, se tivermos alguma tendência ou padrão (formam um cone, funil), há heterocedasticidade

sns.scatterplot(x=y_pred.reshape(-1), y=residuos_std.reshape(-1))
plt.axhline(y=0)

: 

Linearidade: há um indício de valores mais altos saírem do range esperado

Homogeneidade das variâncias: não, é perceptível a presença de padrões fora da reta (heterocedasticidade)

In [ ]:
# checar se resíduos seguem uma distribuição normal
# QQ plot, que avalia se uma amostra segue uma distribuição normal
import pingouin as pg
pg.qqplot(residuos_std, dist='norm', confidence=0.95)
plt.xlabel('Quantis Teóricos')
plt.ylabel('Resíduos na escala padrão')
plt.show()

: 

In [ ]:
# teste de normalidade - shapiro wilk
# h0 - resíduos seguem uma dist. normal
# h1 - resíduos não seguem uma dist. normal
# se o p-value > 0.05 não rejeitamos h0, senão rejeitamos
stat_shapiro, p_value_shapiro = shapiro(residuos.reshape(-1))
stat_shapiro, p_value_shapiro

: 

In [ ]:
# teste de normalidade - kolmogorov-smirnov
# h0 - resíduos seguem uma dist. normal
# h1 - resíduos não seguem uma dist. normal
# se o p-value > 0.05 não rejeitamos h0, senão rejeitamos
stat_ks, p_value_ks = kstest(residuos.reshape(-1), 'norm')
stat_ks, p_value_ks

: 

## Fazendo predições com o modelo

In [ ]:
# se eu estudar 30.4 horas, qual a pontuação prevista pelo modelo?
reg_model.predict([[30.4]])

: 

In [ ]:
# quantas horas estudar para obter 600 pontos (pelo modelo)?
# y = ax + b
# y - b = ax
# (y - b) / a = x
# x = (y - b) / a
(600 - reg_model.intercept_[0]) / reg_model.coef_[0][0]

: 

## Salvar modelo para usar depois

In [ ]:
import joblib
joblib.dump(reg_model, './modelo_regressao.pkl')

: 